In [1]:
 !pip install transformers

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
pip install openpyxl

     |████████████████████████████████| 242 kB 5.0 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx


In [5]:
df=pd.read_excel('/kaggle/input/6-class-bert-preprocessed-dataset/Bert_Preprocessed_Dataset.xlsx')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6405 entries, 0 to 6404
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6405 non-null   int64 
 1   ID          6404 non-null   object
 2   Text        6404 non-null   object
 3   Sentiment   6405 non-null   object
 4   Label       6405 non-null   int64 
 5   Token       6405 non-null   object
 6   Token_ID    6405 non-null   object
dtypes: int64(2), object(5)
memory usage: 350.4+ KB


In [6]:
pip install ktrain

     |████████████████████████████████| 25.3 MB 5.3 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 22.3 MB 47.5 MB/s            
     |████████████████████████████████| 981 kB 56.2 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 263 kB 56.6 MB/s            
     |████████████████████████████████| 2.8 MB 45.6 MB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 468 kB 58.3 MB/s            
     |████████████████████████████████| 3.3 MB 33.7 MB/s            
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for ktrain: filename=ktrain-0.30.0-py3-none-any.wh

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [8]:
import ktrain
from ktrain import text

In [9]:
train=df[:int(.8*len(df))]
test=df[int(.8*len(df)):int(.9*len(df))]
validation=df[int(.9*len(df)):]
x_train=train['Text'].tolist()
y_train=train['Label'].tolist()
x_validation=validation['Text'].tolist()
y_validation=validation['Label'].tolist()
x_test=test['Text'].tolist()
y_test=test['Sentiment'].tolist()

In [10]:
x_train= np.array(x_train)
x_validation= np.array(x_validation)
x_test= np.array(x_test)


In [11]:
categories = ['angry','disgust','fear','happy','sad','surpise']

In [12]:
model_name = 'csebuetnlp/banglabert'
trans = text.Transformer(model_name,maxlen=50,class_names=categories)

Downloading:   0%|          | 0.00/586 [00:00<?, ?B/s]

In [13]:
train = trans.preprocess_train(x_train,y_train)
valid = trans.preprocess_test(x_validation,y_validation)

preprocessing train...
language: bn
train sequence lengths:
	mean : 16
	95percentile : 44
	99percentile : 102


Downloading:   0%|          | 0.00/119 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/528k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 16
	95percentile : 49
	99percentile : 96


In [14]:
model = trans.get_classifier()

404 Client Error: Not Found for url: https://huggingface.co/csebuetnlp/banglabert/resolve/main/tf_model.h5
/opt/conda/lib/python3.7/site-packages/ktrain/text/preprocessor.py:1070: UserWarning: Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. You will need PyTorch installed for this.
  warnings.warn('Could not find Tensorflow version of model.  Attempting to download/load PyTorch version as TensorFlow model using from_pt=True. ' +\


Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [15]:
learner = ktrain.get_learner(model, train_data=train,val_data=valid, batch_size=12)

In [16]:
from sklearn.utils import class_weight
class_weight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
weight={i:class_weight[i] for i in range(6)}

In [17]:
learner.fit_onecycle(2e-5,20,class_weight=weight) 



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/20
427/427 [==============================] - 1321s 3s/step - loss: 1.7826 - accuracy: 0.2172 - val_loss: 1.7440 - val_accuracy: 0.3307
Epoch 2/20
427/427 [==============================] - 1275s 3s/step - loss: 1.6736 - accuracy: 0.4079 - val_loss: 1.4928 - val_accuracy: 0.4509
Epoch 3/20
427/427 [==============================] - 1280s 3s/step - loss: 1.4864 - accuracy: 0.4926 - val_loss: 1.3949 - val_accuracy: 0.4805
Epoch 4/20
427/427 [==============================] - 1294s 3s/step - loss: 1.3113 - accuracy: 0.5570 - val_loss: 1.3800 - val_accuracy: 0.4462
Epoch 5/20
427/427 [==============================] - 1293s 3s/step - loss: 1.1054 - accuracy: 0.6243 - val_loss: 1.3888 - val_accuracy: 0.4711
Epoch 6/20
427/427 [==============================] - 1275s 3s/step - loss: 0.8361 - accuracy: 0.7201 - val_loss: 1.5734 - val_accuracy: 0.4665
Epoch 7/20
427/427 [==============================] - 1277s 3s/step - los

In [18]:
learner.validate(class_names=categories)

              precision    recall  f1-score   support

       angry       0.57      0.58      0.58       137
     disgust       0.23      0.15      0.18        72
        fear       0.12      0.17      0.14        30
       happy       0.79      0.72      0.75       195
         sad       0.42      0.58      0.48       132
     surpise       0.42      0.28      0.34        75

    accuracy                           0.52       641
   macro avg       0.42      0.41      0.41       641
weighted avg       0.53      0.52      0.52       641



array([[ 80,  11,   9,   2,  27,   8],
       [ 26,  11,   2,   9,  19,   5],
       [  4,   1,   5,   2,  13,   5],
       [  8,   7,   6, 141,  30,   3],
       [ 14,  11,   9,  14,  76,   8],
       [  9,   7,   9,  11,  18,  21]])

In [19]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [20]:
y_pred = predictor.predict(x_test)
print(y_pred)

['angry', 'disgust', 'angry', 'angry', 'fear', 'disgust', 'sad', 'fear', 'happy', 'happy', 'disgust', 'angry', 'happy', 'happy', 'angry', 'happy', 'happy', 'surpise', 'angry', 'happy', 'sad', 'fear', 'happy', 'sad', 'angry', 'happy', 'happy', 'angry', 'angry', 'fear', 'happy', 'surpise', 'fear', 'happy', 'angry', 'angry', 'sad', 'angry', 'happy', 'happy', 'sad', 'angry', 'surpise', 'happy', 'angry', 'sad', 'sad', 'sad', 'sad', 'angry', 'sad', 'angry', 'disgust', 'happy', 'angry', 'disgust', 'sad', 'disgust', 'disgust', 'surpise', 'surpise', 'sad', 'angry', 'happy', 'sad', 'surpise', 'fear', 'sad', 'sad', 'sad', 'happy', 'sad', 'happy', 'happy', 'disgust', 'happy', 'sad', 'disgust', 'fear', 'happy', 'disgust', 'angry', 'happy', 'disgust', 'angry', 'happy', 'happy', 'happy', 'surpise', 'happy', 'sad', 'angry', 'happy', 'happy', 'disgust', 'sad', 'sad', 'sad', 'sad', 'happy', 'happy', 'angry', 'sad', 'sad', 'surpise', 'surpise', 'happy', 'sad', 'angry', 'sad', 'happy', 'angry', 'sad', 'su

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.67      0.57      0.61       144
     disgust       0.25      0.20      0.22        66
        fear       0.19      0.19      0.19        32
       happy       0.81      0.82      0.81       187
         sad       0.46      0.59      0.52       148
     surpise       0.35      0.30      0.32        63

    accuracy                           0.56       640
   macro avg       0.45      0.44      0.45       640
weighted avg       0.56      0.56      0.56       640

